In [ ]:
# import libraries
import requests
import os
import hashlib
import pathlib
import json
from datetime import datetime
import pytz
from urllib.parse import urlparse
import time
import pandas as pd
import googlemaps
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# import functions
from functions.routes_api_functions import Location
from functions.routes_api_functions import calculateRoute
from functions.routes_api_functions import parseDataframeToLocations
from functions.routes_api_functions import createDataFrameFromResults
from functions.routes_api_functions import cache_file_path
from functions.routes_api_functions import save_to_cache
from functions.routes_api_functions import load_from_cache
from functions.routes_api_functions import fetch_all_routes
from functions.routes_api_functions import create_departure_time

In [ ]:
# import data
od_300 = pd.read_csv('../Data/routeresultadded_od300.csv')

In [ ]:
origins = od_300[['oid', 'oy', 'ox']]
destinations = od_300[['did', 'dy', 'dx']]

In [ ]:
# convert origins, destinations to geodataframe
origins = gpd.GeoDataFrame(origins, geometry=[Point(xy) for xy in zip(origins.ox, origins.oy)])
destinations = gpd.GeoDataFrame(destinations, geometry=[Point(xy) for xy in zip(destinations.dx, destinations.dy)])

In [ ]:
# apply parsedataframetoLocations
originsLoc = parseDataframeToLocations(origins)
destinationsLoc = parseDataframeToLocations(destinations)

In [ ]:
# api key
key = 'Jessica API Key'

In [ ]:
# test of create_departure_time function
x = [create_departure_time(departure_time=(2023, 11, 27, hour, 0, 0)) for hour in od_300['hour_of_day']]
x[0]

In [ ]:
# test of calculateRoute function
result = calculateRoute(apikey=key, orig=originsLoc[0], dest=destinationsLoc[0], traffic='TRAFFIC_AWARE', departure_time=(2023, 11, 20, od_300['hour_of_day'][0], 0, 0))
result

In [ ]:
departure_times = [(2023, 11, 27, hour, 0, 0) for hour in od_300['hour_of_day']]
# use departure_times list as input
results = fetch_all_routes(apikey=key, origins=originsLoc, destinations=destinationsLoc, traffic='TRAFFIC_AWARE', departure_times=departure_times)

In [ ]:
results

In [ ]:
len(results)

In [ ]:
df = createDataFrameFromResults(results)
df

In [ ]:
new_dataframe = pd.concat([od_300, df], axis=1)
new_dataframe